In [1]:
from RayTrace import RayTrace
from Thickness import Thickness
from Gradient import Gradient

import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
from astropy.io import fits
from copy import deepcopy

import os

os.environ["OMP_NUM_THREADS"] = "10"

In [2]:
gradFile = "./LaserOutput/fl02_X0Y0Z0_21072022_laserscan.fits"
thickFile = "./ThicknessOutput/Tile_2.fits"

In [3]:
myRay = RayTrace()
myRay.loadRadiator(thickFile)

In [4]:
hdul = fits.open(gradFile)
hdul.info()
h = np.array(hdul["H"].data)
v = np.array(hdul["H"].data)

hh, vv = np.meshgrid(h,v, indexing="ij")

xdata = np.array(hdul["XPROJ"].data)
ydata = np.array(hdul["YPROJ"].data)
xdata_err = np.array(hdul["XPROJERR"].data)
ydata_err = np.array(hdul["yPROJERR"].data)

x0 = hdul[0].header["X0"]
y0 = hdul[0].header["Y0"]
thetax0 =  hdul[0].header["THETAX0"]
thetay0 =  hdul[0].header["THETAY0"]


Filename: ./LaserOutput/fl02_X0Y0Z0_21072022_laserscan.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      12   ()      
  1  H             1 ImageHDU         7   (19,)   int64   
  2  V             1 ImageHDU         7   (19,)   int64   
  3  XPROJ         1 ImageHDU         8   (19, 19)   float64   
  4  XPROJERR      1 ImageHDU         8   (19, 19)   float64   
  5  YPROJ         1 ImageHDU         8   (19, 19)   float64   
  6  YPROJERR      1 ImageHDU         8   (19, 19)   float64   
  7  XTHETA        1 ImageHDU         8   (19, 19)   float64   
  8  YTHETA        1 ImageHDU         8   (19, 19)   float64   


In [5]:
def getChi2(params):

    xtrace = np.zeros(hh.shape)
    ytrace = np.zeros(hh.shape)
    i_ray = deepcopy(myRay)

    for i in range(hh.shape[0]):
        for j in range(hh.shape[1]):
            xtrace[i,j], ytrace[i,j] = i_ray.getProjection(params, 0, 0, thetax0, thetay0, hh[i,j], vv[i,j])

    del i_ray

    xymask = (xdata_err > 0 ) & (ydata_err > 0 )
    xchi2 = np.sum(((xtrace - xdata)**2 / xdata_err**2)[xymask])
    ychi2 = np.sum(((ytrace - ydata)**2 / ydata_err**2)[xymask])

    return xchi2 + ychi2


In [6]:
indexMap = myRay.indexMap

In [7]:
chi2 = getChi2(indexMap)

/tmp/ipykernel_34696/3397081692.py:14: RuntimeWarning: divide by zero encountered in true_divide
  xchi2 = np.sum(((xtrace - xdata)**2 / xdata_err**2)[xymask])
/tmp/ipykernel_34696/3397081692.py:15: RuntimeWarning: divide by zero encountered in true_divide
  ychi2 = np.sum(((ytrace - ydata)**2 / ydata_err**2)[xymask])


In [8]:
-0.5 * chi2

-377500995.42436075

In [9]:
def log_prior(params):
    if (params[0] < 1) or (params[8] < 0):
        return -np.inf
    return 0

def log_probability(params):

    # params = [p0, p1, p2, p3, p4, p5, p6, p7, p8]
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    chi2 = getChi2(params)
    if not np.isfinite(chi2):
        return -np.inf
    return -0.5* chi2

In [10]:
pos = indexMap + 1e-6 * np.random.randn(32, 9)
nwalkers, ndim = pos.shape



In [11]:
from multiprocessing import Pool
import time


nsteps = 5000
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, pool=pool)
    start = time.time()
    sampler.run_mcmc(pos, nsteps, progress=True)
    end = time.time()
    multi_time = end - start
    print("Multiprocessing took {0:.1f} seconds".format(multi_time))
    # print("{0:.1f} times faster than serial".format(serial_time / multi_time))

/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value 

In [ ]:
# for p in pos:
#     c = getChi2(p)
#     print (p, c)

/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value 

[ 1.14999991e+00 -5.59657912e-06 -4.23016873e-06 -4.29434449e-07
 -4.96570950e-06 -3.94845005e-06 -4.34069659e-07 -4.16079666e-07
 -1.24868812e-06] nan


/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value 

[ 1.14999911e+00 -6.43097103e-06 -6.59227759e-06 -5.08071859e-07
 -5.05032497e-06 -5.75821542e-06  3.03412910e-08 -1.84525449e-06
  2.62492155e-07] 855354097.975642


/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value 

...

[ 1.15000008e+00 -6.45853027e-06 -5.39354422e-06 -1.34377913e-06
 -4.06941459e-06 -4.79423265e-06 -1.43636904e-07 -1.49924946e-06
  4.35798350e-07] 975998685.2415543


/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value 

[ 1.15000027e+00 -4.77907879e-06 -6.13646651e-06  3.35489449e-07
 -5.96956546e-06 -7.17789907e-06  5.92723495e-07 -1.58933427e-06
  5.07022730e-08] 765765927.5117748


/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value encountered in arcsin
  theta2 = np.arcsin( n1 * np.sin(theta1) / n2 )
/mnt/raid1/HELIX_Data/HELIXGradient/RayTrace.py:71: RuntimeWarning: invalid value 

[ 1.14999907e+00 -5.88185967e-06 -4.33871062e-06  1.28732124e-07
 -4.66818930e-06 -5.78446979e-06 -1.38614839e-06 -1.14105995e-06
  6.62403492e-07] nan
